# Reading the Climate variables

In the following we load the climate variables computed at `./wetter/climate_vars.ipynb`. We index all the rows based on their `nuts` location and the timestamp of the datapoint. This will allow us to merge it with the SWB data later on.

To know from which climate table we want to merge the data, the nuts information has to be provided

In [1]:
# relevant imports
import pandas as pd
import numpy as np
from collections import defaultdict
from sqlalchemy import create_engine, inspect

Reading the climate data:

In [2]:
c_path = "wetter/prod/climate.db"
c_engine = create_engine("sqlite:///"+c_path, echo=False)

In [3]:
# function for reading only the required data
def load_climate_data(con, nuts_lvl):
    # read tables in db
    insp = inspect(con)
    tables = insp.get_table_names()
    tables = [x for x in tables if x[0] == str(nuts_lvl)]
    # create dataframe
    df = pd.DataFrame()
    for table in tables:
        tmp = pd.read_sql_table(table, con)
        tmp["table_name"] = table
        if df.empty:
            df = tmp
        else:
            df = pd.concat([df, tmp], ignore_index=True)
    return df

# Reading the SOEP Household data

Because the individual questionair file is quite big we will merge it chunkwise with the weather and household data. Therefore we first read the hh files. All the required hh data can be found in two different data sets. These are merged in the following:

In [4]:
hbrutto_path = "./soep/SOEP-CORE.v37eu_CSV/CSV/hbrutto.csv"
hl_path = "./soep/SOEP-CORE.v37eu_CSV/CSV/hl.csv"

`hbrutto` contains meta data about the interviews. This includes data such as "day of interview", "location", etc. The `hl` file contains the interview responses of all waves in a long format.

In the following are the columns of interest with in the `hl` and `hbrutto` file with the corresponding meaning. It is written in a dictionary format so we can rename the variables into something more understandable.

In [5]:
hl_var = {
    "syear":"year", # survey year -> prim. key
    "hid":"hid", # hh id -> prim. key
    # hh control variables:
    "hlc0005_h":"hh_einkommen", #[de] Monatliches HH-Netto-Einkommen [harmonisiert]
    "hlc0043":"hh_children" # Number Children
}

hbrutto_var = {
    "syear":"year", # survey year -> prim. key
    "hid":"hid", # hh id -> prim. key
    "bula_h":"sloc" # location -> bundesland/kreis/etc
    #_:"sloc" ...
}

The two files are now read in to their respective dataframe.

In [6]:
df_hl = pd.read_csv(hl_path, usecols=hl_var.keys())
df_hl.rename(hl_var, axis=1, inplace=True)

In [7]:
df_hbrutto = pd.read_csv(hbrutto_path, usecols=hbrutto_var.keys())
df_hbrutto.rename(hbrutto_var, axis=1, inplace=True)

The two dataframes can now be merged on their primary key. According to the documentation of the db the prim. keys are `hid` and `syear`.

In [8]:
df_hh = df_hl.merge(df_hbrutto, how="inner", on=["hid", "year"])

# Reading the SOEP Individual Data

In [9]:
# clear db
!echo > ./prod/data.db

In [10]:
engine = create_engine("sqlite:///prod/data.db", echo=False)

Now we read the individual data. This dataset includes the target variable *SWB* as well as many other control variables. The datasets that contain the information used in this analysis are `ppathl` (tracking file) and `pl` (data)

In [11]:
ppathl_path = "./soep/SOEP-CORE.v37eu_CSV/CSV/ppathl.csv"
pl_path = "./soep/SOEP-CORE.v37eu_CSV/CSV/pl.csv"

In [12]:
ppathl_var = {
    "pid":"pid", # person id -> prim. key
    "syear":"year", #survey year -> prim. key
    # relevant covariates
    "sex":"gender", # Current life satisfaction [0-10] TARGET VARIABLE
    "gebjahr":"birth_year", # year of birth
    "partner":"relationship", # [0] no partner, [1] spouse, [2] partner, [3] Probably spouse
}

pl_var = {
    # ids
    "pid":"pid", # person id -> prim. key
    "hid":"hid", # hh id -> forg. key
    "syear":"year", #survey year -> prim. key
    "ptagin":"day", #day of interview
    "pmonin":"month", #month of interview
    # target variable
    "plh0182":"swb", # Current life satisfaction [0-10]
    # relevant covariates
    "plh0171":"health", # Current Health [1-5] (0=schlecht, 10=gut)
    "plb0021":"unemployed", # [2] No [1] Yes
    "plh0173":"work", # [0-10] not satisfied <-> very satisfied, NOTE: many nan - family?
    "plh0174":"work_hh", # same as above (NOTE: maybe take max of both?)
    "plg0030":"education", # Total Education, Training Item Nonresponse, NOTE: many nan
    "plh0175":"income_satisfaction" # Satisfaction With Household Income
}

The `ppathl` contains the tracking data of a person. This includes for instance the age or marital status.

In [13]:
df_ppathl = pd.read_csv(ppathl_path, usecols=ppathl_var.keys())
df_ppathl.rename(ppathl_var, axis=1, inplace=True)

The `pl` file is the largest in the database. Most machines will not support loading this file into memory. Fortunatiely, a left join needs to be performed on this file such that it is possible to do the merger chunkwise.

In [14]:
chunksize = 1000
nuts_lvl = 1
df_climate = load_climate_data(c_engine, nuts_lvl=nuts_lvl) 
for chunk in pd.read_csv(pl_path, chunksize=chunksize, usecols=pl_var.keys()):
    # rename vars
    chunk.rename(pl_var, axis=1, inplace=True)
    
    ## MERGE WITH OTHER DATASETS
    # merge with tracking data
    chunk = chunk.merge(df_ppathl, on=["year", "pid"], how="inner")
    # merge with household
    chunk = chunk.merge(df_hh, on=["year", "hid"], how="inner")
    
    ## CALCULATE RELEVANT VARIABLES
    # age:
    chunk["age"] = chunk["year"] - chunk["birth_year"]
    # time stamp:
    chunk["time"] = pd.to_datetime(chunk[['year', 'month', 'day']], errors='coerce')
    # drop unuseful columns:
    chunk.drop(['year', 'month', 'day'], axis=1, inplace=True)
    # delete invalid time stamps as they cannot be merged with climate data:
    chunk = chunk[chunk['time'].notna()]
    
    ## MERGE WITH CLIMATE DF
    final = pd.merge(chunk, df_climate, on=["time", 'sloc'], how='inner')
    
    ## SAVE TO DATABASE
    final.to_sql(f"{nuts_lvl}_data", con = engine, if_exists='append')